## This notbeook computes the idealized search limits for a given network of future detectors
#### User needs to pass results file (.hdf) containing the following for a population of simulated injections
* Network SNR
* Parameter values -- mchirp & eccentricity

### Obtain the efficiency factor (fraction of measurable and detectable injections) for a given detector configuration

In [1]:
import numpy as np, glob,h5py,dynesty,corner,time
import pycbc
import numpy as np
from pycbc import filter, psd, waveform, types, conversions, cosmology
import astropy 
from scipy.interpolate import interp1d
from scipy.interpolate import griddata
import scipy
import glob
import pickle
import h5py
from scipy.interpolate import interp1d, griddata, RegularGridInterpolator, Rbf
from decimal import Decimal
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True})
from matplotlib.colors import Normalize
import seaborn as sns
import matplotlib as mpl
from matplotlib import rc



sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')

mpl.rcParams['figure.figsize'] = [20.0, 7.0]
plt.rcParams['figure.dpi'] = 600
mpl.rcParams['font.size'] = 18

rc('font', family='serif', weight = 'bold')
rc('text', usetex=True)

colors = sns.color_palette("husl", 8)

In [20]:
models = ['belzynscki', 'sedda', 'trani', 'fragione']
model_labels = ['Belczynski et al. (2018a) (globular cluster)',
                'Arca Sedda (2020b) (globular cluster)',
               'Trani et al. (2021) (triples)',
               'Fragione et al. (2019) (nuclear cluster)']

#det_list = ['O3', 'aplus', 'asharp', 'ET', 'CE', 'CE-ET']
det_list = ['asharp']

local_merger_rate_dict = {'asharp': [5000, 5000, 5000, 5000],
                          'CE': [275,275,275,275],
                          }

### Sensitive volume for fixed detectable eccentricity or as a function of the parameter space?
#### Change the value of detectable_ecc 
#### change it to 1e-10 to get limits corresponding to the full population

In [21]:
fixed_ecc = True
print('Fixed eccentricity: ', fixed_ecc)

if fixed_ecc:
    detectable_ecc = 1e-10
    snr_threshold = 10.0
    
    upper_limit = {}
    sens_volume = {}
    for det in det_list:
        print(det)
        label_ind = 0
        
        for model in models:
            results_file = '{}/results/{}.hdf'.format(model, det)
            local_merger_rate = local_merger_rate_dict[det][label_ind]
            
            hf = h5py.File(results_file,'r')
           
            e = np.sort(hf['valid/eccentricity'][:])
            det_mchirp = np.array(hf['valid/mchirp'][:])
            snrs = np.array(hf['valid/sigma'][:])

            inde = e > detectable_ecc
            inde = np.array(inde)

            nsources = len(np.where(inde == True)[0])
            ndetected = len(np.where(snrs[inde] > snr_threshold)[0])
            detected_ind = snrs[inde] > snr_threshold

            print(model, 'Total', len(det_mchirp), 'constrained sources : ', nsources, 'ndetected: ', ndetected, 'Vol: ',
                  float(ndetected)/local_merger_rate*0.682, 'max det_mchirp detected: ',  max(det_mchirp[inde]), ndetected/len(det_mchirp))           

            upper_limit[model_labels[label_ind]] = local_merger_rate * 2.303 / float(ndetected) 
            label_ind += 1

Fixed eccentricity:  True
asharp
belzynscki Total 5000000 constrained sources :  5000000 ndetected:  23715 Vol:  3.2347260000000007 max det_mchirp detected:  6.4483790372662355 0.004743
sedda Total 5000000 constrained sources :  5000000 ndetected:  108163 Vol:  14.753433200000002 max det_mchirp detected:  21.646700386896057 0.0216326
trani Total 5000000 constrained sources :  5000000 ndetected:  147880 Vol:  20.170832 max det_mchirp detected:  24.164718623412636 0.029576
fragione Total 5000000 constrained sources :  5000000 ndetected:  107305 Vol:  14.636402 max det_mchirp detected:  18.31918543468221 0.021461


In [22]:
print('Limits for systems with a fixed eccentricity threshold of :', detectable_ecc)
print(upper_limit)

Limits for systems with a fixed eccentricity threshold of : 1e-10
{'Belczynski et al. (2018a) (globular cluster)': 0.48555766392578537, 'Arca Sedda (2020b) (globular cluster)': 0.10645969508981815, 'Trani et al. (2021) (triples)': 0.0778671896131999, 'Fragione et al. (2019) (nuclear cluster)': 0.10731093611667676}


## Now we compute limits for measurably eccentric systems
#### $\mathcal{M}_c$ and $e_{10}$ grid points as a function of SNR are required for this operation. We will then interpolate the 3d volume btw these points and also extrpolate the function to estimate the fraction of injections recovered

In [23]:
def get_3d_interpolant(e_dict, mc_dict):
    x_array = np.array([])
    y_array = np.array([])
    z_array = np.array([])
    for key in e_dict.keys():
        e = e_dict[key]
        mc = mc_dict[key]

        x = np.linspace(np.log10(min(e_dict[key])), np.log10(max(e_dict[key])), 500)
        interp = interp1d(np.log10(e), mc, fill_value='extrapolate', kind='linear')
        snr = np.full(x.shape, float(key))
        #plt.scatter(np.log10(e), mc, marker='x')

        x_array = np.concatenate([x_array, x])
        y_array = np.concatenate([y_array, interp(x)])
        z_array = np.concatenate([z_array, snr])
        #ax.scatter(x, interp(x), snr, s=60, c='k')
        #plt.plot(x, interp(x), label=float(key))

    xx, yy = np.meshgrid(np.linspace(np.log10(1e-4),np.log10(0.2),50), np.linspace(1.0,70.0,50))
    zz = griddata((x_array, y_array), z_array, (xx,yy), method='linear')


    rbf3 = Rbf(x_array, y_array, z_array, function='multiquadric', smooth=4)
    zz = rbf3(xx,yy)
    return rbf3

def read_results(filename, det):
    hf = h5py.File(filename)
    if det == '3-O3':
        mass1 = np.array(hf['mass1'][:])
        mass2 = np.array(hf['mass2'][:])
        eccentricity = np.array(hf['eccentricity'][:])
        sigma = np.array(hf['sigma'][:])
    else:
        mass1 = np.array(hf['valid/mass1'][:])
        mass2 = np.array(hf['valid/mass2'][:])
        eccentricity = np.array(hf['valid/eccentricity'][:])
        sigma = np.array(hf['valid/sigma'][:])
            
    mchirp = conversions.mchirp_from_mass1_mass2(mass1, mass2)
    return eccentricity, mchirp, sigma

In [14]:
e_file = '../Fischer-analysis/asharp-results-e.pkl'
mc_file = '../Fischer-analysis/asharp-results-mc.pkl'
print('e grid: ', e_file, '\n mc grid:', mc_file)

with open(e_file, 'rb') as efile:
    e_dict = pickle.load(efile)
with open(mc_file, 'rb') as mcfile:
    mc_dict = pickle.load(mcfile)
    
rbf3 = get_3d_interpolant(e_dict, mc_dict)

e grid:  ../Fischer-analysis/asharp-results-e.pkl 
 mc grid: ../Fischer-analysis/asharp-results-mc.pkl


In [15]:
det = 'asharp'
snr_threshold = 10.0

#models = ['sedda','trani','fragione']
#models = ['belzynscki']
upper_limit_measurable = {}

label_ind = 0
for model in models:
    results_file = '{}/results/{}.hdf'.format(model, det)
    #results_file = 'belzynscki/results/scaled-2-O3.hdf'
    local_merger_rate = local_merger_rate_dict[det][label_ind]
    
    e_temp, mc_temp, sigma_temp = read_results(results_file, det)
    sort_i = np.argsort(sigma_temp)
    ind = sort_i[np.where(sigma_temp[sort_i] > snr_threshold)[0]]

    sigma = sigma_temp[ind]
    eccentricity = e_temp[ind]
    mchirp = mc_temp[ind]

    count = 0
    measured_ind = []
    for k in range(len(eccentricity)):
        interp_snr = rbf3(np.log10(eccentricity[k]), mchirp[k])
        if sigma[k] > interp_snr:
            count += 1
            measured_ind.append(k)
    
    frac_sources = float(count)/len(e_temp)               
    upper_limit_measurable[model_labels[label_ind]] = float("{:.3e}".format(2.303*local_merger_rate/count))
    print(model, 'Detectable:', len(eccentricity), 'Measurable: ', count, 'Total: ', len(e_temp))
    label_ind += 1

belzynscki Detectable: 23715 Measurable:  9 Total:  5000000
sedda Detectable: 108163 Measurable:  1350 Total:  5000000
trani Detectable: 147880 Measurable:  15614 Total:  5000000
fragione Detectable: 107305 Measurable:  43749 Total:  5000000


In [19]:
print(upper_limit_measurable)

{'Belczynski et al. (2018a) (globular cluster)': 1279.0, 'Arca Sedda (2020b) (globular cluster)': 8.53, 'Trani et al. (2021) (triples)': 0.7375, 'Fragione et al. (2019) (nuclear cluster)': 0.2632}
